# Digesting Data from NGINX Access Logs

This notebook was created as an exercise in data manipulation and visualization in Python. The data used in this notebook consists of the last two months of searches on [GitHub Search](https://seart-ghs.si.usi.ch), as recorded in the NGINX access logs. The raw logs were parsed and enriched with geographical data, which was retrieved from [ipinfo](https://ipinfo.io). Our end-goal is to gain more insight into the usage of the aforementioned platform. Before we begin, we need to import all the necessary library utilities.

In [ ]:
from ast import literal_eval

from geopandas import read_file
from geopandas.datasets import get_path
from matplotlib.pyplot import bar, figure, pie, show, subplot
from pandas import DataFrame
from pandas import concat, json_normalize, read_csv
from shapely.geometry import Point

With that out of the way, we can now read the CSV file, and convert the string representation of the JSON data into Python dictionaries.

In [ ]:
df = read_csv("access.csv", header=None, names=["access", "city"], converters={0: literal_eval, 1: literal_eval})
df.head()

Next, let's transform our two-column `DataFrame` into a single `DataFrame` with the JSON data expanded into columns. Let's start with the `city` column:

In [ ]:
mappings = {
  "name": "city",
  "country_continent_name": "continent_name",
  "country_continent_code": "continent_code",
  "position_latitude": "latitude",
  "position_longitude": "longitude",
}
city = json_normalize(df.city, sep="_").rename(columns=mappings)
coordinate_mapper = lambda row: Point(row.longitude, row.latitude)
city["coordinates"] = city.apply(coordinate_mapper, axis=1)
city.drop(columns=["latitude", "longitude"], inplace=True)
city = city[["city", "region", "country_name", "country_code", "continent_name", "continent_code", "coordinates"]]
city

Let's do the same for the `access` column:

In [ ]:
access = json_normalize(df.access, sep="_", max_level=0)
access = access.join(json_normalize(access["user_agent"], sep="_"))
access.drop(columns=["user_agent"], inplace=True)
access = access[["ip", "time", "query", "status", "size", "browser", "os", "device", "referer"]]
access

Merging the two frames together:

In [ ]:
df = DataFrame.join(access, city)
df

We can now initiate our analysis. Let's start off easy by just calculating the _total_ number of requests per day.

In [ ]:
date_only = lambda ts: ts.split("T")[0]
days_accessed = df.time.map(date_only)
daily_visitors = days_accessed.value_counts().sort_index()
daily_visitors.plot(figsize=(20, 10))

Seems like the platform has been accessed every day. Let's break this 60-day period down into a statistical summary:

In [ ]:
daily_visitors.describe()

As we can see, the average number of daily searches is around 25, with the minimum being as low as only 2 searches in a day, and the maximum being more than 100 searches in a single day! Keep in mind that we are only looking at the number of requests made on a daily basis. This means that the number of unique visitors is likely to be lower. Over the last two months, the total number of unique IP addresses that have accessed the platform is:

In [ ]:
df.ip.nunique()

And counting unique visitors per day:

In [ ]:
ip_access = df[['time', 'ip']]
ip_access.loc[:, "time"] = ip_access.loc[:, "time"].map(date_only)
ip_access_by_day = ip_access.groupby("time").nunique()
ip_access_by_day.plot(figsize=(20, 10))

And on average:

In [ ]:
ip_access_by_day.describe()

Having gathered insights on daily traffic, we can now move on calculating the distribution of traffic by continent and country.

In [ ]:
def get_top_n(data_frame: DataFrame, n: int = 10):
  data_frame_counts = data_frame.value_counts()
  data_frame_top = data_frame_counts.head(n - 1)
  data_frame_other_count = data_frame_counts.tail(1 - n).sum()
  data_frame_other = DataFrame(data={"count": [data_frame_other_count]}, index=["Other"])
  return concat([data_frame_top, data_frame_other])

continent_top = get_top_n(df.continent_name, 5)
country_top = get_top_n(df.country_name)

autopct = "%1.1f%%"
figure(figsize=(20, 10))
subplot(1, 2, 1)
pie(list(continent_top["count"]), labels=continent_top.index, autopct=autopct)
subplot(1, 2, 2)
pie(list(country_top["count"]), labels=country_top.index, autopct=autopct)
show()

While the distribution of traffic by continent is relatively balanced and simple to comprehend, the distribution of traffic by country appears to be more complex. More than a quarter of the traffic is widely distributed across a large number of countries. For this reason, plotting the data on a map might be more insightful. To do that, we must first load some geographical geometry data:

In [ ]:
country_geometry = read_file(get_path("naturalearth_lowres"))
country_geometry = country_geometry[country_geometry.continent != "Antarctica"]
country_geometry = country_geometry[country_geometry.pop_est > 0]
country_geometry = country_geometry[["name", "geometry"]]
country_geometry = country_geometry.rename(columns={"name": "country_name"})
country_geometry = country_geometry.set_index("country_name")
country_geometry

Next, let's join the geometry data of each individual country with the traffic data.

In [ ]:
country_count = df.country_name.value_counts()
world_count = country_geometry.join(country_count)
world_count

Finally, we can plot the data on a map.

In [ ]:
world_count.plot(
  figsize=(20, 10),
  column="count",
  cmap="viridis",
  legend=True,
  missing_kwds={
    "color": "lightgrey",
    "label": "No Data",
    "edgecolor": "red",
  },
  legend_kwds={
    "label": "Traffic",
    "shrink": 0.65,
  },
)

From this we can not only see all the countries that the platform has been accessed from, but also the countries that it has not been accessed from. The US is of particular interest, as it has no recorded traffic. On the flipside, small amounts of trafic were recorded in Africa (Morocco and Egypt), as well as South America (Brazil and Chile). 

Shifting our focus to a different angle, we can now analyze the distribution of traffic by browser, operating system, and device. To start, let's first isolate the relevant columns into a smaller `DataFrame`:

In [ ]:
user_agent = df[["browser", "os"]]
user_agent

Notice that both `browser` and `os` contain versioning information, which may potentially lead to more numerous unique values. To reduce the values to their respective families, we can apply a simple transformation:

In [ ]:
browsers = user_agent[["browser"]].map(lambda x: x.rsplit(' ', 1)[0])
operating_systems = user_agent[["os"]].map(lambda x: x.rsplit(' ', 1)[0])
user_agent = user_agent.assign(os=operating_systems, browser=browsers)
user_agent

With the data in a more manageable state, we can now calculate the distribution of traffic by browser and operating system:

In [ ]:
browsers_top = get_top_n(user_agent.browser, 5)
operating_systems_top = get_top_n(user_agent.os, 5)

figure(figsize=(20, 10))
subplot(1, 2, 1)
bar(browsers_top.index, browsers_top["count"])
subplot(1, 2, 2)
bar(operating_systems_top.index, operating_systems_top["count"])
show()

To no one's surprise, Chrome reigns supreme in the browser market, closely followed by Edge and Firefox. Although we develop and test our platform UI exclusively on Chrome, the fact that a third of the platform visits comes from other browsers puts an emphasis on the need for cross-browser compatibility, which would be ensured through end-to-end testing. While manual testing would be too time-consuming, automated testing would be a more efficient solution. Although you might be asking yourself why we are considering operating systems for a web application, the answer lies in the fact that the platform is also accessible through mobile devices. As such, we made sure that the design of the site was responsive. While the data shows that a minute fraction of the traffic comes from mobile devices, it's important to keep in mind that a good first impression from a handheld can convince users to try the desktop version as well.

To conclude our analysis, let's take a look at what users are searching for, starting with the most popular query parameters.
Note that for this analysis, we will not consider:
- `page` and `per_page`, as they are used for pagination
- `nameEquals`, as it is always included in the query
Furthermore, the ranged parameters (e.g. `starsMin` and `starsMax`) will be grouped under a single umbrella (i.e. `stars`).

In [ ]:
blacklist = ["page", "per_page", "nameEquals"]
query_parameters = df["query"].map(lambda query: query.keys())
query_parameters_by_query = query_parameters.explode()
query_parameters_by_query = query_parameters_by_query.loc[~query_parameters_by_query.isin(blacklist)]
query_parameters_by_query = query_parameters_by_query.str.removesuffix("Min")
query_parameters_by_query = query_parameters_by_query.str.removesuffix("Max")
query_parameters_by_query_count = query_parameters_by_query.value_counts()
query_parameters_by_query_count.plot.bar(figsize=(20, 10))

To no particular surprise, we can see that `language` comes out on top, followed closely by `stars`, `name` and `commits`. What is interesting however is that the `topic` parameter placed in the top 10, in spite of being a relatively new addition to the dataset. This could be an indication that the feature is being well received by the users. On the flipside, exact match  parameters such as `label` and `license` have seen significantly less use. This is most likely due to the fact that there is currently no way to specify multiple values for a single query, which i my humble opinion is a feature we should consider adding in the future.

Given that the `language` parameter is the most popular, it would be interesting to see which languages users search for the most. To do this, we can extract the `language` parameter values and calculate their distribution:

In [ ]:
def get_languages(query: dict):
  if "language" in query:
    first, = query["language"]
    return first.lower()
  return None

query_parameter_language = df["query"].map(get_languages).dropna()
query_parameter_language_count = get_top_n(query_parameter_language, 4)
figure(figsize=(15, 7))
pie(query_parameter_language_count["count"], labels=query_parameter_language_count.index, autopct=autopct)
show()

As was expected, `Java`, `Python` and `JavaScript`, the three of the most popular programming languages in the world constitute a cumulative 65% of language-specific searches. While looking at the top does not reveal anything particularly surprising, it's at the very bottom where some new insights are obtained:

In [ ]:
least_popular_languages = query_parameter_language.value_counts().tail()
for item in list(least_popular_languages.keys()):
    print(item)

The first thing that stands out is the fact that `Lua` and `Haskell` were both queried. Although we do not mine these languages, it's worth considering adding them to the list of future targets. The second thing that stands out is the fact that `dart, kotlin` and `c/c++` were search terms. Echoing the sentiment from the previous section, it's worth considering adding the ability to search for multiple languages at once, allowing sampling on a set of languages.